# Calibration of Black and Scholes, Merton, Kou, Variance Gamma parameters
This notebook aims to find the optimal parameters of **Black-Scholes**, **Merton Jump Diffusion**, **Kou Jump Diffusion** and **Variance Gamma** models. To do so, we compute the european option prices using **closed formulas**, available for all the 4 models, and the **Fast Fourier Transform** for the VG model. Given these theoretical prices, the **implied volatilities** are computed comparing them with real market prices, minimizing their difference. Then we estimate the additional parameters of each model, using the python module `scipy.optimize`.

*reference: https://github.com/cantaro86/Financial-Models-Numerical-Methods/tree/master*


In [1]:
from functions.MERTONpricer import Merton_pricer
from functions.BSpricer import BS_Pricer
from functions.KOUpricer import Kou_pricer
from functions.VGpricer import VG_pricer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy as scp
import scipy.stats as ss
import scipy.optimize as scpo
import time

random.seed(100)

Let's start retrieving historical prices for **european call** and **put** options starting from date **2016-01-20** and expiring **1 year** later. All the data have been collected from [OptionsDX](https://www.optionsdx.com/shop/) and preprocessed in `plainvanilla.py` module. Only options such that
$$ K = (1 \pm 0.2) * S0 $$ have been selected.

In [2]:
df_call = pd.read_csv('data/options_spx_call_OTM_2016.csv')
df_put = pd.read_csv('data/options_spx_put_OTM_2016.csv')
print(f'NUM OF CALL OPTS: {df_call.shape[0]}')
print(df_call.head(5))
print(f'NUM OF PUT OPTS: {df_put.shape[0]}')
print(df_put.tail(5))

NUM OF CALL OPTS: 16
   QUOTE_DATE  UNDERLYING_LAST EXPIRE_DATE   C_BID   C_ASK  STRIKE     C_IV
0  2016-01-20          1859.48  2017-01-20  128.49  131.80  1875.0  0.20726
1  2016-01-20          1859.48  2017-01-20  115.50  118.60  1900.0  0.20243
2  2016-01-20          1859.48  2017-01-20  103.00  106.10  1925.0  0.19893
3  2016-01-20          1859.48  2017-01-20   91.20   94.29  1950.0  0.19460
4  2016-01-20          1859.48  2017-01-20   80.20   83.10  1975.0  0.18961
NUM OF PUT OPTS: 16
    QUOTE_DATE  UNDERLYING_LAST EXPIRE_DATE   P_BID   P_ASK  STRIKE     P_IV
11  2016-01-20          1859.48  2017-01-20  124.40  127.61  1750.0  0.22459
12  2016-01-20          1859.48  2017-01-20  133.01  136.30  1775.0  0.21925
13  2016-01-20          1859.48  2017-01-20  142.29  145.61  1800.0  0.21460
14  2016-01-20          1859.48  2017-01-20  152.01  155.41  1825.0  0.20960
15  2016-01-20          1859.48  2017-01-20  162.39  165.89  1850.0  0.20556


The dataframes **df_calls** and **df_put** contain both 83 options sorted by strike price. For our purpose, using the $25\%$ of these DFs is enough and can lead to very good results. Thus, we sample the rows and then reformulate the dataframes adding **Midpoint** and **Spread** columns.

In [3]:
calls = df_call.sample(frac=1, replace=False).sort_index().reset_index(drop=True)
puts = df_put.sample(frac=1, replace=False).sort_index().reset_index(drop=True)

calls['C_Midpoint'] = abs(calls['C_BID'] + calls['C_ASK']) / 2
puts['P_Midpoint'] = abs(puts['P_BID'] + puts['P_ASK']) / 2
calls['C_Spread'] = calls['C_BID'] - calls['C_ASK']
puts['P_Spread'] = puts['P_BID'] - puts['P_ASK']

q = 0           # dividend yield
r = 0.02        # risk-free interest rate
sigma = 0.2     #volatility (variance of diffusion process)
S0 = calls.iloc[0]['UNDERLYING_LAST']
T = 1           # time-to-maturity
call_strikes = calls['STRIKE']    # array of K for call options
put_strikes = puts['STRIKE']      # array of K for put options
exercise = 'european'

call_prices = calls['C_Midpoint']
put_prices = puts['P_Midpoint']

The following code snippet initializes objects of class *BS_pricer*, *Merton_pricer*, *Kou_pricer*, *VG_pricer* with default values as parameters. Then it computes the theoretical call prices using closed formulas of each 4 models, with strike prices given by the *call_strikes* array. Additionally, we use the **Midpoint** price as the option market prices.P

In [4]:
BS = BS_Pricer(S0=S0, r=r, q = q, sigma=sigma, ttm=T, exercise=exercise, K=None)
Merton = Merton_pricer(S0=S0, K=None, ttm=T, r=0.05, q = q, sigma=0.12, lambd=0.5, meanJ=-0.1, stdJ=0.2, exercise=exercise)
Kou = Kou_pricer(S0=S0, K=None, ttm=T, r=r, sigma=0.12, lambd=0.5, p=0.6, eta1=10, eta2=5, exercise=exercise)
VG = VG_pricer(S0, K=None, ttm=T, r=r, q=q, sigma=0.15, theta=-0.2, nu=0.2, exercise=exercise)

call_th_prices = pd.DataFrame(columns=['Strike','BlackScholes', 'Merton', 'Kou', 'VarianceGamma'])
call_th_prices['Strike'] = call_strikes

for i, K in enumerate(call_strikes):
    bs = BS.closed_formula_call(K)
    mert = Merton.closed_formula_call(K)
    kou = Kou.closed_formula_call(K)
    vg = VG.closed_formula_call(K)
    call_th_prices.iloc[i, 1:] = [bs, mert, kou, vg]

print(f'Theoretical call options prices:')
print(call_th_prices.head(5)) #print(call_th_prices.tail(4))

Theoretical call options prices:
   Strike BlackScholes      Merton         Kou VarianceGamma
0  1875.0   158.311855  173.930733  132.529771    139.953942
1  1900.0   146.789271  159.818815  120.361619    129.363913
2  1925.0   135.905931  146.455396  109.088286    119.499129
3  1950.0   125.647782  133.855507   98.691309    110.324748
4  1975.0   115.998703  122.027395   89.144379    101.804905


Same for put prices.

In [5]:
put_th_prices = pd.DataFrame(columns=['Strike','BlackScholes', 'Merton', 'Kou', 'VarianceGamma'])
put_th_prices['Strike'] = put_strikes

for i, K in enumerate(put_strikes):
    bs = BS.closed_formula_put(K)
    mert = Merton.closed_formula_put(K)
    kou = Kou.closed_formula_put(K)
    vg = VG.closed_formula_put(K)
    put_th_prices.iloc[i, 1:] = [bs, mert, kou, vg]

print(f'Theoretical put options prices:')
print(put_th_prices.head(5))

Theoretical put options prices:
   Strike BlackScholes     Merton        Kou VarianceGamma
0  1475.0    16.251289  18.081857  17.316044      5.143545
1  1500.0    19.492529   20.37145  19.309027      6.929789
2  1525.0    23.191207  22.886964  21.570424      9.185867
3  1550.0    27.379887  25.648285  24.145247     11.984084
4  1575.0    32.089391  28.678076  27.083658     15.394203


## Implied volatility
The function belows implements $3$ methods to compute implied volatility: [Newton](https://en.wikipedia.org/wiki/Newton%27s_method) method, the [Bisection](https://en.wikipedia.org/wiki/Bisection_method) method and a more advanced one, named [Brent](https://en.wikipedia.org/wiki/Brent%27s_method) method. Apart from the initial guess, there is no substantial difference in the final result between **Newton** and **bisection** methods (*fsolve*). The **Implied Volatility** is that value $\sigma$ that must be inserted into the Black-Scholes (BS) formula in order to retrieve the option price quoted in the market:
    $$ BS(S, K, T, r, \sigma) = P,  $$
where $S$ is the underlying spot price, $K$ is the strike, $T$ time to maturity, $r$ risk-free interest rate and $P$ the option price quoted in the market. All these quantities are **observable**.
   

In [6]:
def implied_volatility(price, S, strike, t, rate, q, type_o, method='fsolve', disp=True ):
    """ Returns Implied volatility
        methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return BS.BlackScholes(type_o=type_o, S0=S, K=strike, ttm=t, r=rate, q=q, sigma=vol) - price

    def vega(vol):
        return BS.vega(S, strike, rate, q, vol, t)

    if method =='fsolve':
        X0 = [0.01, 0.2, 0.35, 7]        #initial guess points for imp.vol.
        for x_0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x_0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if disp:
        return -1

The following code snippet computes the implied volatility of **call** and **put** options market prices.

In [7]:
IV_market_c = []
for i in range(len(call_prices)):
    IV_market_c.append(implied_volatility(call_prices[i], S=S0, strike=call_strikes[i], t = T, rate=0.02, q = q, type_o='call', method='fsolve'))

print(f'Implied volatilities of market prices (calls):\nS0 = {S0}')
for a,b in zip(call_strikes.tail(6), IV_market_c[-6:]):
    print(f'K = {a}, IV = {round(b, 4)}')

IV_market_p = []
for i in range(len(put_prices)):
    IV_market_p.append(implied_volatility(put_prices[i], S=S0, strike=put_strikes[i], t = T, rate=0.01, q = q, type_o='put', method='fsolve'))

print(f'Implied volatilities of market prices (puts):\nS0 = {S0}')
for a,b in zip(put_strikes.head(6), IV_market_p[:6]):
    print(f'K = {a}, IV = {round(b, 4)}')

Implied volatilities of market prices (calls):
S0 = 1859.48
K = 2125.0, IV = 0.1372
K = 2150.0, IV = 0.1346
K = 2175.0, IV = 0.1318
K = 2200.0, IV = 0.1295
K = 2225.0, IV = 0.1269
K = 2250.0, IV = 0.1248
Implied volatilities of market prices (puts):
S0 = 1859.48
K = 1475.0, IV = 0.3001
K = 1500.0, IV = 0.2958
K = 1525.0, IV = 0.2915
K = 1550.0, IV = 0.2869
K = 1575.0, IV = 0.2828
K = 1600.0, IV = 0.2787


##  Weighted Calibration (call options)
Let's step now into the calibration of model parameters.
If we define $\Theta$ the set of parameters, the goal is to find the optimal parameters $\Theta^*$ that minimize the following objective function:
$$ \sum_{i=1}^{N} w_i \biggl( P_i - f(K_i|\Theta) \biggr)^2 $$
where $w_i$ are weights, usually defined as
$$ w_i = \frac{1}{\text {spread}_i },$$ $P_i$ are the market prices and $f$ is the pricing function. In our case $f$ is given by **Merton** Jump Diffusion model, **Kou** Jump Diffusion model, or **Variance Gamma** process. To perform this optimization problem, many numerical methods can be used. In particular, we test two functions of `scipy.optimize`:
1. **curve_fit**, a least-squares curve fitting method which works with bounds. The default algorithm is [Trust Region Reflective (trf)](https://en.wikipedia.org/wiki/Trust_region). The [Levemberg-Marquadt](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm) has been tried as well, to test the optimization problem without setting boundaries, but the results don't make any sense.
2. **Least-Squares**, a constrained minimization problem which uses Trust region reflective method by default. This method is the most indicated to solve the non-linear least squares optimization problem of our purpose.
All the optimizations are carried out by initializing a starting point as the array $x_0 = [params]$ and setting feasible bounds.


In [8]:
call_spreads = calls['C_Spread']

### Black and Scholes model
The only unknown parameter to calibrate in Black and Scholes model is the **implied volatility**, $\sigma$. Thus, we minimize the difference between the computed theoretical prices and the market prices of call options.

In [9]:
x0 = 0.5
bounds = [1e-5, 2]

def f_BlackScholes_call(x, sigm):
    BS = BS_Pricer(S0=S0, K = x, ttm=T, r=r, q=0, sigma=sigm, exercise='call')
    return BS.closed_formula_call(x)

res1_calls = scpo.curve_fit(f_BlackScholes_call, call_strikes, call_prices, p0 = x0, bounds=bounds, sigma= call_spreads)
sigw_c = round(res1_calls[0][0],4)

In [10]:
def cost_function(x, strikes, mkt_prices):
    sigma = x
    BS = BS_Pricer(S0=S0, K = None, ttm=T, r=r, q=0, sigma=sigma, exercise='put')
    sq_err = np.sum( call_spreads * (BS.closed_formula_call(strikes) - mkt_prices)**2)
    return sq_err

result_c = scpo.least_squares(cost_function, x0, args=(call_strikes, call_prices), bounds=bounds, method = 'trf', verbose=1)
opt_sigma_c = result_c.x[0]

`ftol` termination condition is satisfied.
Function evaluations 23, initial cost 3.6752e+12, final cost 1.5332e+06, first-order optimality 6.49e+02.


In [11]:
print('METHOD 1: CURVE_FIT (trf)')
print(f'> Calibrated Volatility from Calls [σ] = {sigw_c} \t {round(sigw_c*100,2)}%')
print('METHOD 2: LEAST-SQUARES (trf)')
print(f'> Calibrated Volatility from Calls [σ] = {opt_sigma_c} \t {round(opt_sigma_c*100,2)}%')

METHOD 1: CURVE_FIT (trf)
> Calibrated Volatility from Calls [σ] = 0.1449 	 14.49%
METHOD 2: LEAST-SQUARES (trf)
> Calibrated Volatility from Calls [σ] = 0.14907304408670605 	 14.91%


### Merton Jump Diffusion
The Merton Jump diffusion ones are the volatility $\sigma$, the Poisson rate of jumps $\lambda$, the mean rate of jump intensity $m$ and its variance rate $v$, assuming that the intensity of jumps follows a *Normal distribution*.


In [12]:
x0 = [0.10,  0.93, -0.097,  0.092]  # initial guess: [σ, λ, m, v]
bounds = ( [1e-3, 1e-2, -10, 1e-5], [2, 10, 10, 5] )

In [13]:
def f_Mert(x, sigma, lambd, meanJ, stdJ):
    Mert = Merton_pricer(S0=S0, K=x, ttm=T, r=r, q=0, sigma=sigma, lambd=lambd, meanJ=meanJ, stdJ=stdJ, exercise=exercise)
    return Mert.closed_formula_call(x)

start1=time.time()
mert1 = scpo.curve_fit(f_Mert, call_strikes, call_prices, p0=x0, bounds=bounds, sigma=call_spreads)
end1=time.time()

mert_params1 = [round(p,4) for p in mert1[0][:4]]

##### Method 2. Least-squares

In [14]:
x0 = [0.4, 1, -0.1, 0.2]      # initial guess: [σ, λ, m, v]
bounds = ( [1e-3, 1e-2, -10, 0], [2, np.inf, 10, 5] )

def cost_function(x, strikes, mkt_prices):
    sigma, lambd, meanJ, stdJ = x
    M = Merton_pricer(S0, None, T, r, q, sigma, lambd, meanJ, stdJ, exercise)
    sq_err = np.sum( call_spreads*(M.closed_formula_call(strikes) - mkt_prices)**2)
    return sq_err

start2 = time.time()
mert2 = scpo.least_squares(cost_function, x0, args=(call_strikes, call_prices), bounds=bounds, method = 'trf', verbose=2)
end2 = time.time()

mert_params2 = [round(p,4) for p in mert2.x[:4]]

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9277e+12                                    3.25e+13    
       1              2         1.5295e+12      3.98e+11       3.15e-01       1.40e+14    
       2              3         5.1173e+11      1.02e+12       1.35e-01       2.67e+13    
       3              4         4.5573e+11      5.60e+10       2.50e-01       3.01e+13    
       4              5         2.9113e+11      1.65e+11       6.38e-02       1.37e+13    
       5              6         1.7927e+11      1.12e+11       1.15e-01       3.42e+12    
       6              7         1.1329e+11      6.60e+10       6.80e-02       3.53e+12    
       7              9         8.4249e+10      2.90e+10       4.34e-02       5.28e+11    
       8             10         4.4165e+10      4.01e+10       2.28e-02       3.52e+11    
       9             11         1.3843e+10      3.03e+10       8.44e-02       1.59e+12    

In [28]:
print('WEIGHTED OPT: CURVE_FIT (trf)')
print(f'> Calibrated Volatlity [σ] = {round(mert1[0][0],4)} \t {round(mert1[0][0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(mert1[0][1],2))
print('> Calibrated Jump Mean = ', round(mert1[0][2],2))
print('> Calibrated Jump St. dev.  = ', round(mert1[0][3],5))
print(f'ELAPSED TIME: {end1-start1} sec')

print('\nMETHOD 1: LEAST SQUARES (trf)')
print(f'> Calibrated Volatlity [σ] = {mert_params2[0]} \t {round(mert_params2[0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(mert_params2[1],2))
print('> Calibrated Jump Mean = ', round(mert_params2[2],3))
print('> Calibrated Jump St. dev.  = ', round(mert_params2[3],3))
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

WEIGHTED OPT: CURVE_FIT (trf)
> Calibrated Volatlity [σ] = 0.0713 	 7.13%
> Calibrated Jump intensity [λ] =  0.74
> Calibrated Jump Mean =  -0.17
> Calibrated Jump St. dev.  =  1e-05
\ELAPSED TIME: 40.197970151901245 sec

METHOD 1: LEAST SQUARES (trf)
> Calibrated Volatlity [σ] = 0.0878 	 8.78%
> Calibrated Jump intensity [λ] =  0.91
> Calibrated Jump Mean =  -0.118
> Calibrated Jump St. dev.  =  0.096
TIME ELAPSED:  307.53 sec


In [16]:
print(mert_params1)
print(mert_params2)

[0.0713, 0.7368, -0.1748, 0.0]
[0.0878, 0.9085, -0.118, 0.0963]


### Kou Jump Diffusion


In [17]:
x0 = [0.1, 1, 0.4, 5, 5] # initial guess: [σ, λ, p, η_1, η_2]
bounds = ( [1e-3, 1e-2, 0, 0, 0], [2, 20, 0.99,  15, 15] )

##### Method 1. TRF (Bounds)

In [18]:
def f_Kou(x, sigma, lambd, p, eta1, eta2):
    KouJD = Kou_pricer(S0=S0, K=x, ttm=T, r=r, sigma=sigma, lambd=lambd, p=p, eta1=eta1, eta2=eta2, exercise=exercise)
    return KouJD.closed_formula_call(x)


start1 = time.time()
kou1 = scpo.curve_fit(f_Kou, call_strikes, call_prices, p0=x0, bounds=bounds, sigma=call_spreads)
end1 = time.time()

kou_params1 = [round(p,4) for p in kou1[0][:5]]

##### Method 2. LEAST SQUARES (With Bounds)

In [19]:
# Define the objective function
def cost_function(x, strikes, mkt_prices):
    sigm, lamb, p, eta1, eta2 = x
    KOU = Kou_pricer(S0=S0, K=strikes, ttm=T, r=r, sigma=sigm, lambd=lamb, p=p, eta1=eta1, eta2=eta2, exercise=exercise)
    sq_err = np.sum(call_spreads*(KOU.closed_formula_call(strikes) - mkt_prices) ** 2)
    return sq_err

start2=time.time()
kou2 = scpo.least_squares(cost_function, x0, args=(call_strikes, call_prices),  method='trf', bounds=bounds, verbose=2)
end2=time.time()

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.3386e+10                                    3.97e+11    
       1              2         3.1468e+08      3.31e+10       8.06e+00       7.13e+09    
       2              3         1.6676e+06      3.13e+08       2.00e+00       5.70e+08    
       3              6         6.3606e+05      1.03e+06       3.67e-02       4.36e+06    
       4              7         5.8223e+05      5.38e+04       3.85e-03       3.89e+07    
       5             10         5.7501e+05      7.22e+03       1.27e-03       7.74e+06    
       6             11         5.7213e+05      2.88e+03       1.14e-03       4.79e+06    
       7             12         5.6647e+05      5.66e+03       1.66e-03       2.73e+06    
       8             13         5.5523e+05      1.12e+04       2.53e-03       4.23e+06    
       9             14         5.3325e+05      2.20e+04       6.26e-03       2.57e+06    

In [20]:
kou_params2 = [round(p,4) for p in kou2.x[:5]]

In [21]:
print('WEIGHTED OPT: CURVE_FIT (trf)')
print(f'> Calibrated Volatlity [σ] = {kou_params1[0]} \t {kou_params1[0] * 100}%')
print('> Calibrated Jump intensity [λ] = ', kou_params1[1])
print(f'> Calibrated Upward Jump probability [p] = {kou_params1[2]}, [q] = {round(1 - kou_params1[2], 2)}')
print('> Calibrated Rate of Exp. 1  [η_1] = ', kou_params1[3])
print('> Calibrated Rate of Exp. 2  [η_2] = ', kou_params1[4])
print(f'TIME ELAPSED: {end1-start1} sec')

print('METHOD 2: Least-squares')
print(f'> Calibrated Volatlity [σ] = {round(kou_params2[0],4)} \t {round(kou_params2[0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(kou_params2[1],2))
print(f'> Calibrated Upward Jump probability [p] = {round(kou_params2[2],2)}, [q] = {round(1-kou_params2[2],2)}')
print('> Calibrated Rate of Exp. 1  [η_1] = ', round(kou_params2[3],2))
print('> Calibrated Rate of Exp. 2  [η_2] = ', round(kou_params2[4],2))
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

WEIGHTED OPT: CURVE_FIT (trf)
> Calibrated Volatlity [σ] = 0.053 	 5.3%
> Calibrated Jump intensity [λ] =  3.0035
> Calibrated Upward Jump probability [p] = 0.0, [q] = 1.0
> Calibrated Rate of Exp. 1  [η_1] =  15.0
> Calibrated Rate of Exp. 2  [η_2] =  15.0
TIME ELAPSED: 258.18559741973877 sec
METHOD 2: Least-squares
> Calibrated Volatlity [σ] = 0.07 	 7.0%
> Calibrated Jump intensity [λ] =  0.93
> Calibrated Upward Jump probability [p] = 0.25, [q] = 0.75
> Calibrated Rate of Exp. 1  [η_1] =  13.05
> Calibrated Rate of Exp. 2  [η_2] =  6.56
TIME ELAPSED:  3713.37 sec


In [22]:
print(kou_params1)
print(kou_params2)

[0.053, 3.0035, 0.0, 15.0, 15.0]
[0.07, 0.9296, 0.2498, 13.0534, 6.5574]


### Variance Gamma


In [23]:
x0 = [0.1, -0.2, 0.1]   # initial guess: [σ, θ, v]
bounds = ( [1e-2, -10, 0], [5, 10, 10] )

##### Method 1. CURVE FIT (Bounds)

In [24]:
def f_VG(strikes, sigmax, thetax, nux):
    VGamma = VG_pricer(S0=S0, K=None, ttm=T, r=r, q=0, sigma=sigmax, theta=thetax, nu=nux, exercise=exercise)
    vg_prices = []
    for k in strikes:
        vg_prices.append(VGamma.closed_formula_call(k))
    return vg_prices

start1 = time.time()
vg1 = scpo.curve_fit(f_VG, call_strikes, call_prices, p0=x0, bounds=bounds, sigma=call_spreads)
end1 = time.time()

vg_params1 = [round(p,4) for p in vg1[0][:3]]

##### Method 2. LEAST-SQUARES (Trust Region Reflective, Bounds)

In [25]:
def cost_function(x, strikes, mkt_prices):
    sigma, theta, nu = x
    VG = VG_pricer(S0, None, T, r, q, sigma, theta, nu, exercise)
    prices = []
    for k in strikes:
        prices.append(VG.closed_formula_call(k))
    sq_err = np.sum(call_spreads*(prices - mkt_prices)**2)
    return sq_err

start2=time.time()
vg2 = scpo.least_squares(cost_function, x0, args=(call_strikes, call_prices),  method='trf', bounds=bounds, verbose=2)
end2=time.time()

vg_params2 = [round(p,4) for p in vg2.x[:3]]

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.1400e+08                                    5.53e+10    
       1              3         1.1308e+07      1.03e+08       5.94e-02       8.67e+09    
       2              4         3.4128e+06      7.90e+06       1.27e-02       1.94e+08    
       3              7         3.2846e+06      1.28e+05       7.18e-03       9.55e+07    
       4              8         3.2103e+06      7.43e+04       8.03e-03       1.30e+08    
       5              9         3.1401e+06      7.02e+04       1.57e-02       6.20e+08    
       6             11         3.1316e+06      8.54e+03       2.91e-03       1.95e+08    
       7             12         3.1072e+06      2.44e+04       8.43e-04       1.84e+08    
       8             13         3.0637e+06      4.35e+04       1.71e-03       1.64e+08    
       9             14         2.9958e+06      6.80e+04       3.56e-03       1.26e+08    

In [26]:
print('WEIGHTED OPT: CURVE_FIT (trf)')
print(f'> Calibrated Volatlity [σ] = {vg_params1[0]}, \t {round(vg_params1[0]*100,2)}%')
print('> Calibrated mean rate gamma process [θ] = ', vg_params1[1])
print('> Calibrated variance rate gamma process [v]= ', vg_params1[2])
print(f'TIME ELAPSED:  {round(end1-start1,2)} sec')

print('METHOD 1: LEAST-SQUARES (trf)')
print(f'> Calibrated Volatlity [σ] = {vg_params2[0]}, \t {round(vg_params2[0]*100,2)}%')
print('> Calibrated mean rate gamma process [θ] = ', vg_params2[1])
print('> Calibrated variance rate gamma process [v]= ', vg_params2[2])
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

WEIGHTED OPT: CURVE_FIT (trf)
> Calibrated Volatlity [σ] = 0.071, 	 7.1%
> Calibrated mean rate gamma process [θ] =  0.2872
> Calibrated variance rate gamma process [v]=  0.2969
TIME ELAPSED:  40.2 sec
METHOD 1: LEAST-SQUARES (trf)
> Calibrated Volatlity [σ] = 0.1259, 	 12.59%
> Calibrated mean rate gamma process [θ] =  0.1921
> Calibrated variance rate gamma process [v]=  0.3751
TIME ELAPSED:  307.53 sec


In [27]:
print(vg_params1)
print(vg_params2)

[0.071, 0.2872, 0.2969]
[0.1259, 0.1921, 0.3751]


### Monte Carlo Option Pricing and Calibration
The following code snippet aims to find the best parameters for each model, minimizing the difference between the monte carlo prices and the market prices of **call** and **put** options.

#### Black and Scholes Model

In [29]:
# Define the objective function
def obj_function(x, strikes, mkt_prices, days=252, N=100):
    sigm = x
    BS = BS_Pricer(S0,r,q,sigm, T, exercise=exercise, K=None)
    bs_prices = []
    for k in strikes:
        bs_prices.append(BS.MonteCarlo_Call(k,251, days, N))
    diff = bs_prices - mkt_prices
    return np.sum(diff**2)

additional_args = (call_strikes, call_prices)

x0 = 0.20

# METHOD SEQUENTIAL LEAST SQUARES
res_bs = scpo.minimize(obj_function, x0, args=additional_args,  method='BFGS')


In [30]:
sigma_opt = res_bs.x
print(sigma_opt)

[0.2]
